In [ ]:
# Tutorial 7：Modeling the visual field (with FilterNet)

In [ ]:
import numpy as np
from bmtk.builder import NetworkBuilder

# 构建LGN网络
net = NetworkBuilder('lgn')  # Initialize network called 'lgn'

# 构建OFF细胞（对撤光敏感）
net.add_nodes(
    N=20, 
    model_type='virtual',
    model_template='lgnmodel:tOFF_TF15',
    x=np.random.uniform(0.0, 240.0, 20),
    y=np.random.uniform(0.0, 120.0, 20),
    spatial_size=1.0,
    dynamics_params='tOFF_TF15.json'
)

# 构建ON细胞（对给光敏感）
net.add_nodes(  # add 10 simple ON cells
    N=20, 
    model_type='virtual',
    model_template='lgnmodel:tON',
    x=np.random.uniform(0.0, 240.0, 20),
    y=np.random.uniform(0.0, 120.0, 20),
    spatial_size=1.0,
    dynamics_params='tON_TF8.json'
)

net.build()
net.save_nodes(output_dir='sim_ch07/network')

In [ ]:
from bmtk.utils.sim_setup import build_env_filternet

# 设置环境
build_env_filternet(
    base_dir='sim_ch07', 
    network_dir='sim_ch07/network', 
    tstop=3000.0,
    include_examples=True)

In [ ]:
# 通过修改simulation_config.json文件给予不同的刺激
'''
    1. 定制的影像刺激（movie）：npy格式的文件，以三维矩阵的形式存储：(时间戳, X坐标, y坐标)
    {"inputs": {
         "movie_input": {
             "input_type": "movie",
             "module": "movie",
             "data_file": "/path/to/my/movie.npy",  # 视频文件路径
             "frame_rate": 1000.0                   # 每秒帧数
             }}}
    
    
    2. 光栅刺激（grating）
    {"inputs": {
         "LGN_spikes": {
             "input_type": "movie",
             "module": "graiting",
             "row_size": 120,                       # 屏幕宽度
             "col_size": 240,                       # 屏幕高度
             "gray_screen_dur": 0.5,                # 在光栅启动前的若干秒钟内显示可选的灰色屏幕（默认值：0.0）
             "cpd": 0.04,                           # 空间频率，单位周期/度（默认值：0.05）
             "temporal_f": 4.0,                     # 时间频率，单位赫兹（默认值：4.0）
             "contrast": 0.8,                       # 最大对比度，必须在0到1.0之间（默认值：1.0）
             "theta": 45.0,                         # 方位角，单位度（默认值：45.0）
             "phase": 0.0,                          # 时间相位，单位度（默认值：0.0）
             "frame_rate": 1000.0                   # 每秒帧数
             }}}


    3. 全场闪光（full field flash）
    {"inputs": {
        "full_field_flash": {
            "input_type": "movie",
            "module": "full_field_flash",
            "row_size": 120,                        # 屏幕宽度
            "col_size": 240,                        # 屏幕高度
            "t_on": 1000.0,                         # 从开始到启动闪光灯的时间，单位毫秒
            "t_off": 2000.0,                        # 闪光时长，单位毫秒
            "max_intensity": 20.0                   # 与灰屏相比，闪光时屏幕的亮度（大于0为亮，小于0为暗）
            "frame_rate": 1000.0                    # 每秒帧数
            }}}
    
    4. 黑色区域（looming）：一个黑色区块从画面中心慢慢向四周扩张（在PPT里放一张图片，添加入场动画-形状-效果选项选放大，就是差不多的效果）
    {"inputs": {
          "LGN_spikes": {
              "input_type": "movie",
              "module": "looming",
              "row_size": 120,                      # 屏幕宽度
              "col_size": 240,                      # 屏幕高度
              "frame_rate": 1000.0,                 # 每秒帧数
              "gray_screen_dur": 0.5,               # 初始灰屏持续时间，单位秒
              "t_looming": 1.0                      # loom的时间，单位秒
              }}}
'''

In [ ]:
from bmtk.simulator import filternet

config = filternet.Config.from_json('sim_ch07/config.json')
config.build_env()
net = filternet.FilterNetwork.from_config(config)
sim = filternet.FilterSimulator.from_config(config, net)
sim.run()

In [ ]:
from bmtk.analyzer.spike_trains import plot_raster

_ = plot_raster(config_file='sim_ch07/config.json', group_by='model_template')